In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd gdrive/MyDrive/colab/

/content/gdrive/MyDrive/colab


In [6]:
!ls

dev_seen.jsonl
dev_unseen.jsonl
ernie-vil
finetuned_vilbertcc_election_memes
finetuned_vilbert_election_memes
finetuned_visualbertcoco_election_memes
finetuned_visualbert_election_memes
hateful_and_election_memes_detectron.lmdb
mmf
pretrained_vilbertcc_election_memes
pretrained_vilbert_election_memes
pretrained_visualbertcoco_election_memes
pretrained_visualbert_election_memes
save
saved_models
test_seen.jsonl
test_unseen.jsonl
train_hateful_and_election.jsonl
train.jsonl
XjiOc5ycDBRRNwbhRlgH.zip


In [11]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

In [9]:
true_labels = pd.read_json('dev_seen.jsonl',lines=True).set_index('id')

In [10]:
simple_average = pd.read_csv('ernie-vil/EL365072/EL365072_dev_seen_SA.csv').set_index('id')

In [19]:
joined_scores = true_labels.rename(columns={'label': 'true_label'})[['true_label']] \
    .join(simple_average.rename(columns={'proba':'predicted_prob'})[['predicted_prob']], how='outer') \
    .drop_duplicates()

In [21]:
joined_scores['predicted_label'] = np.where(joined_scores['predicted_prob'] > joined_scores['predicted_prob'].median(), 1, 0)


In [22]:
display(joined_scores)

,true_label,predicted_prob,predicted_label
id,,,
8291,1,1.623416,1
46971,1,-4.213025,0
3745,1,0.240041,1
83745,1,-4.214904,0
80243,1,-1.005365,1
...,...,...,...
83675,0,-4.420274,0
37198,0,-4.491071,0
48670,0,-4.329164,0


In [28]:
np.sum(joined_scores['predicted_label'] == 1)

250

In [31]:
metrics.roc_auc_score(joined_scores['true_label'], joined_scores['predicted_prob'])

0.6966443167816165

In [32]:
metrics.accuracy_score(joined_scores['true_label'], joined_scores['predicted_label'])

0.63

In [34]:
# Regular ernie-vil
regular_ernie_vil_sa = pd.read_csv('ernie-vil/EL365072_dev_seen_SA.csv').set_index('id')

In [35]:
regular_joined_scores = true_labels.rename(columns={'label': 'true_label'})[['true_label']] \
    .join(regular_ernie_vil_sa.rename(columns={'proba':'predicted_prob'})[['predicted_prob']], how='outer') \
    .drop_duplicates()

In [36]:
regular_joined_scores['predicted_label'] = np.where(regular_joined_scores['predicted_prob'] > regular_joined_scores['predicted_prob'].median(), 1, 0)

In [37]:
metrics.roc_auc_score(regular_joined_scores['true_label'], regular_joined_scores['predicted_prob'])

0.7559808612440191

In [38]:
metrics.accuracy_score(regular_joined_scores['true_label'], regular_joined_scores['predicted_label'])

0.686